In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os
from markdown_it import MarkdownIt
import fitz
from requests import get
from bs4 import BeautifulSoup
import re
from brave import Brave

brave = Brave()
gpt4 = ChatOpenAI(model="gpt-4")
turbo = ChatOpenAI(model="gpt-3.5-turbo")
make_prompt = ChatPromptTemplate.from_template
parser = StrOutputParser()

In [4]:
def get_company_name(job_description):
    cname_p = make_prompt("""
    The following is a job description:
    '''
    {job_description}
    '''

    What company is the job at? Respond with the name of the company only.
    """)

    return (cname_p | turbo | parser).invoke({"job_description": job_description})

In [5]:
def get_website_text(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Accept-Language": "en-US,en;q=0.9",
        "Connection": "keep-alive",
        "DNT": "1"
    }

    resp = get(url, headers=headers)
    soup = BeautifulSoup(resp.text)
    for header in soup.find_all('header'):
        header.decompose()

    # Remove elements by class name (e.g., navbars)
    for navbar in soup.find_all(class_='navbar'):
        navbar.decompose()

    # Alternatively, if the navbars use <nav> tags
    for nav in soup.find_all('nav'):
        nav.decompose()
    
    return re.sub(r'\s+', ' ', soup.body.text)[:1000]

In [6]:
def get_info(company_name):
    q = f"'{company_name}' company website"
    search_results = brave.search(q=q, count=5)
    res = ('\n'.join([f'[{i + 1}] {search_results.urls[i].host} {d}' for i, d in enumerate(search_results.descriptions)]))

    nav_prompt = make_prompt("""
    You are doing research about a company called {company_name} so that you can write a cover letter for job there. You search '{q}' and see the following search results.

    {res}

    Which of these links is most promising as a way to learn more about the company, given your task?

    Respond with a single number corresponding to the most promising link. Do not include any words in your response. Your response should be parsable as an integer.
    """)

    model_resp = (nav_prompt | turbo | parser).invoke({"company_name": company_name, "res": res, "q": q})
    try:
        index = int(model_resp)
    except ValueError:
        index = 1
    url = search_results.urls[index - 1]
    
    text = get_website_text(str(url))
    
    sum_prompt = make_prompt("""
    You are doing research on the company {company_name} to help with writing a cover letter. You have come across the following search result.

    URL: {url}

    Webpage text: {text}

    Based on this information, take notes on aspects of the company that you could tie into the cover letter. Respond with a few sentences summarizing the most important points.
    """)

    return (sum_prompt | turbo | parser).invoke({"company_name": company_name, "url": str(url), "text": text})

# get_tie_in(company_name)

In [7]:
def get_mission_statement(company_name):
    q = f"'{company_name}' mission statement"
    search_results = brave.search(q=q, count=5)
    res = ('\n'.join([f'[{i + 1}] {search_results.urls[i].host} {d}' for i, d in enumerate(search_results.descriptions)]))

    nav_prompt = make_prompt("""
    You are doing research about a company called {company_name}. Your current goal is to determine the company's mission statement. You search '{q}' and see the following search results.

    {res}

    Which one of these is the company's website? If multiple links are to the company's website, select the most promising one.

    Respond with a single number corresponding to the most promising link. Do not include any words in your response. Your response should be parsable as an integer.
    """)

    model_resp = (nav_prompt | turbo | parser).invoke({"company_name": company_name, "res": res, "q": q})
    try:
        index = int(model_resp)
    except ValueError:
        index = 1
    url = search_results.urls[index - 1]
    
    text = get_website_text(str(url))
    
    sum_prompt = make_prompt("""
    Try to determine the mission statement of {company_name} based on the following search result.

    URL: {url}

    Webpage text: {text}

    What is the mission statement {company_name}?
    - If you have found the company's mission statement, respond with a direct quote.
    - If you have not found {company_name}'s mission statement, respond with "Unable to determine."
    """)

    return (sum_prompt | turbo | parser).invoke({"company_name": company_name, "url": str(url), "text": text})

get_mission_statement("Google")

'"Our mission is to organize the world’s information and make it universally accessible and useful."'

In [8]:
def make_cv(job_description, resume, info, mission_statement):
    cv_prompt = make_prompt("""
    Your task is to write a cover letter for a job application.

    Here is the job description:
    '''
    {job_description}
    '''

    Here is the applicant's resume:
    '''
    {resume}
    '''

    Here is some additional information about the company:
    '''
    {info}
    '''

    Here is the company's mission statement:
    '''
    {mission_statement}
    '''

    Write a cover letter for the applicant
    - Tailor the cover letter to the job description
    - Draw on one or two specific experiences from the resume that are relevant to the job description. Explain what skills I applied in the experience and how this connects to the needs of the company.
    - Make sure to use important phrases from the job description in the cover letter
    - The cover letter should be no longer than 500 words
    - Ensure that the cover letter is professional and free of grammatical errors.

    Do
    - Use markdown where appropriate
    - Start with a strong opening line
    - Select three skills that you can “immediately bring to the table” and show examples to convince your employer you have those skills
    - Tell stories and show examples that illustrate your skills rather than just listing accomplishments 
    - Be concise (1 page, 3 to 4 paragraphs)
    - Focus on what you can bring to the organization
    - Connect your experiences directly to the role
    - Convey enthusiasm and demonstrate excitement for the specific role and/or organization  

    Don't
    - Pretend to have any skills that you do not
    - Mention every professional experience you have (i.e. don’t reiterate your resume in paragraph form)
    - Focus exclusively on why you’re excited about the role without demonstrating how you’d be a good fit 
    - Focus exclusively on yourself and accomplishments without making it clear how it can help the organization 
    - Make any grammatical errors or typos 
    - Use any special characters

    Here is a template illustrating what a good cover letter might look like:
    '''
    Dear Hiring Manager,

    Start with a short introductory sentence or two that provides a compelling hook. Emphasis why you are passionate about organization/role or what you can bring to the role.  

    More specifically, here’s what I can bring to your organization: 

    Skill or Qualification I: Story illustrating a time you exhibited this skill…. Sentence summarizing how that skill can be applied in the specific role. 

    Skill or Qualification II: Story illustrating a time you exhibited this skill…. Sentence summarizing how that skill can be applied in the specific role. 

    [Optional] Skill or Qualification III: Story illustrating a time you exhibited this skill…. Sentence summarizing how that skill can be applied in the specific role. 


    I am ready for the challenge to [something the role/company is going to address] by bringing/leveraging my experiences and skills in XYZ. [Any additional relevant insight/qualification you want to offer or anything you want to share about your candidacy]. 


    Thank you for your time…

    Sincerely,

    Name
    '''


    Make sure you only return the cover letter and nothing else.

    Most importantly, do not make up any information about the candidate that is not in the resume. 
    Do not mention any skills or qualifications that are not in the resume. Even if 
    the job description says a skill is required, do not mention it in the cover letter if it was not 
    mentioned in the resume. This is the most important part of your job.
    """)

    return (cv_prompt | gpt4 | parser).invoke({
        "job_description": job_description,
        "resume": resume,
        "info": info,
        "mission_statement": mission_statement
        })

In [18]:
def md_to_pdf(s, filename):
    md = MarkdownIt()
    html = md.render(s)

    story = fitz.Story(html=html, user_css='p {font-family: "Times New Roman", Times, serif;}')

    MEDIABOX = fitz.paper_rect("letter")  # output page format: Letter
    WHERE = MEDIABOX + (36, 36, -36, -36)  # leave borders of 0.5 inches
    if filename.endswith(".pdf"):
        filename = filename[:-4]
    writer = fitz.DocumentWriter(f"{filename}.pdf")  # create the writer

    more = 1
    while more:  # loop outputting the story
        device = writer.begin_page(MEDIABOX)  # make new page
        more, _ = story.place(WHERE)  # layout into allowed rectangle
        story.draw(device)  # write on page
        writer.end_page()  # finish page

    writer.close()  # close output file

md_to_pdf(cover_letter, "cover_letter")

In [ ]:
with open("job_description.txt") as jd:
    job_description = jd.read()

with open("resume.txt") as r:
    resume = r.read()

In [ ]:
company_name = get_company_name(job_description)
info = get_info(company_name)
mission_statement = get_mission_statement(company_name)
cover_letter = make_cv(job_description, resume, info, mission_statement)
md_to_pdf(cover_letter, f"{company_name}_cover_letter")